# **Most Common Coordinate Frame Conventions in Robotics** 

## 1. ENU (East–North–Up)

**Axes**

* X → East
* Y → North
* Z → Up



**Used in**

* ROS (default world frame)
* Geographic mapping
* Robotics navigation

**Notes**

* Very intuitive for ground robots
* Z points upward (gravity is −Z)



<img src="images/RPY_angles_of_cars.png"  />



---

## 2. NED (North–East–Down)

**Axes**

* X → North
* Y → East
* Z → Down



**Used in**

* Aerospace
* IMU datasheets
* PX4 / ArduPilot internally

**Notes**

* Gravity is +Z
* Extremely common in flight control
* Opposite vertical direction compared to ENU

<img src="images/frame_heading.px4.png" width="90%" height="90%"  />

---

## 3. Camera Frame (OpenCV Convention)

**Axes**

* X → Right (image columns)
* Y → Down (image rows)
* Z → Forward (out of camera)


**Used in**

* OpenCV
* Computer vision
* Pinhole camera model

**Notes**

* This breaks robotics assumptions
* Must be converted before using with IMU or SLAM
* One of the most common sources of bugs

<img src="images/opencv_coordinate.png"  />


---


# **Most Common Coordinate Frame Conventions in Visualization Software** 

## 1. OpenGL

### World / View Coordinate Convention (de-facto)

**Axes**

* X → Right
* Y → Up
* Z → **Backward** (camera looks toward −Z)


**Camera**

* Camera is at origin
* Looks along **−Z**
* +X right, +Y up

**Clip Space (after projection)**

* X ∈ [−1, +1]
* Y ∈ [−1, +1]
* Z ∈ [−1, +1] (OpenGL)

**Used in**

* OpenGL
* Vulkan (similar, with differences in depth)
* Many 3D engines

### Key Point

OpenGL is **right-handed**, but the **camera looks down −Z**, which often feels inverted to robotics users.



<img src="images/opengl_coordinate.png"  />




---

## 2. RViz (ROS Visualization)

### World Frame (Fixed Frame)

**Axes**

* X → Forward
* Y → Left
* Z → Up


**This is exactly ROS ENU / FLU**

**Used in**

* RViz
* TF trees
* ROS navigation and SLAM

### Camera Behavior

* Orbit camera rotates around Z
* Top-down view looks along −Z
* Grid lies in X-Y plane

### Key Point

If your robot looks sideways or upside-down in RViz, **your TF is wrong**, not RViz.

---

## 3. Rerun

### World Coordinate System

**Axes**

* X → Right
* Y → Forward
* Z → Up

**Right-handed**: ✅ Yes

This is **different from ROS**.

### Camera

* Default camera looks roughly toward origin
* Z is always up
* Y is forward

**Used in**

* Rerun C++ / Python SDK
* SLAM debugging
* Computer vision pipelines


---

## 4. Comparison Summary (Critical)

| System          | X       | Y       | Z        | Forward | Up |
| --------------- | ------- | ------- | -------- | ------- | -- |
| OpenGL          | Right   | Up      | Backward | −Z      | +Y |
| RViz (ROS)      | Forward | Left    | Up       | +X      | +Z |
| Rerun           | Right   | Forward | Up       | +Y      | +Z |
| Camera (OpenCV) | Right   | Down    | Forward  | +Z      | −Y |

---


## What is Forward?
“When I move the robot forward in code, should I change x, y, or z?” Change the axis defined as forward in your current frame

## Common Conversions



Below is a **clean, practical conversion guide** that you can **directly use in code** when your data is:

* **Recorded from OpenCV (camera)**
* **Recorded from PX4 (IMU / pose)**
* **Simulated in Gazebo**

and you want to **display it correctly in ROS (RViz) and Rerun**.



---

# 1. Frame conventions we will use (fixed reference)

### OpenCV camera (optical frame)

```
X → right
Y → down
Z → forward
```

### PX4

* **World**: NED

  ```
  X → North
  Y → East
  Z → Down
  ```
* **Body**: FRD

  ```
  X → forward
  Y → right
  Z → down
  ```

### Gazebo (classic / Ignition default)

```
X → forward
Y → left
Z → up
```

(Same as ROS FLU)

### ROS / RViz

```
X → forward
Y → left
Z → up
```

### Rerun

```
X → right
Y → forward
Z → up
```

---

# 2. OpenCV camera → ROS (RViz)

### Position vector

OpenCV → ROS base_link / camera_link:

```
X_ros =  Z_cv
Y_ros = -X_cv
Z_ros = -Y_cv
```

### Rotation matrix

The fixed rotation is:

$$
R_{\text{ros}\leftarrow\text{cv}} =
\begin{bmatrix}
0 & 0 & 1 \\
-1 & 0 & 0 \\
0 & -1 & 0
\end{bmatrix}
$$

Apply as:

```
R_ros = R_ros_cv * R_cv
p_ros = R_ros_cv * p_cv
```

This is **exactly what ROS optical frames expect**.

---

# 3. OpenCV camera → Rerun

Rerun world:

```
X → right
Y → forward
Z → up
```

OpenCV:

```
X → right
Y → down
Z → forward
```

### Position

```
X_rerun =  X_cv
Y_rerun =  Z_cv
Z_rerun = -Y_cv
```

### Rotation

$$
R_{\text{rerun}\leftarrow\text{cv}} =
\begin{bmatrix}
1 & 0 & 0  \\
0 & 0 & 1  \\
0 & -1 & 0
\end{bmatrix}
$$

---

# 4. PX4 NED → ROS ENU (RViz)

This is **mandatory**.

### Position

```
X_enu =  Y_ned
Y_enu =  X_ned
Z_enu = -Z_ned
```

### Rotation

$$
R_{\text{enu}\leftarrow\text{ned}} =
\begin{bmatrix}
0 & 1 & 0 \\
1 & 0 & 0 \\
0 & 0 & -1
\end{bmatrix}
$$

Use:

```
R_enu = R_enu_ned * R_ned
p_enu = R_enu_ned * p_ned
```

If you skip this:
❌ gravity points up
❌ yaw flips
❌ RViz looks “rotated”

---

# 5. PX4 Body (FRD) → ROS base_link (FLU)

PX4 body:

```
X → forward
Y → right
Z → down
```

ROS:

```
X → forward
Y → left
Z → up
```

### Position / velocity

```
X_ros =  X_frd
Y_ros = -Y_frd
Z_ros = -Z_frd
```

### Rotation

$$
R_{\text{flu}\leftarrow\text{frd}} =
\begin{bmatrix}
1 & 0 & 0  \\
0 & -1 & 0 \\
0 & 0 & -1
\end{bmatrix}
$$

---

# 6. PX4 → Rerun (recommended path)

Do **not** go PX4 → Rerun directly.

✅ Correct pipeline:

```
PX4 (NED, FRD)
   ↓
ROS (ENU, FLU)
   ↓
Rerun
```

This avoids mistakes and keeps physics consistent.

---

# 7. ROS → Rerun

ROS:

```
X → forward
Y → left
Z → up
```

Rerun:

```
X → right
Y → forward
Z → up
```

### Position

```
X_rerun = -Y_ros
Y_rerun =  X_ros
Z_rerun =  Z_ros
```

### Rotation

$$
R_{\text{rerun}\leftarrow\text{ros}} =
\begin{bmatrix}
0 & -1 & 0 \\
1 &  0 & 0 \\
0 &  0 & 1
\end{bmatrix}
$$

---

# 8. Gazebo → ROS

✅ **No conversion needed**

Gazebo already uses:

```
X → forward
Y → left
Z → up
```

Only ensure:

* model frame
* link frame
* sensor frame

are consistent.

---

# 9. Gazebo → Rerun

Just apply **ROS → Rerun** from section 7.

---

# 10. One canonical code pattern (recommended)

Always do this:

```python
# 1. Express motion in body frame
v_body = $$v_forward, 0, 0$$

# 2. Convert body → world
v_world = R_world_body @ v_body

# 3. Convert world → visualization frame
p_vis = R_vis_world @ p_world
R_vis = R_vis_world @ R_world
```

Never mix steps.

---

# 11. Sanity checks (do these every time)

✅ Gravity points down
✅ Forward motion increases expected axis
✅ Yaw rotation spins around up-axis
❌ “It looks right” without math (invalid test)

---

# 12. Final takeaway

* **Sensors never agree on frames**
* **Visualizers never agree on frames**
* **Math always works if frames are explicit**

If you want next, I can:

* Give you **ready-to-use Python/C++ helper functions**
* Apply this **directly to EuRoC cam0/cam1/IMU**
* Show **Rerun + RViz side-by-side correctness checks**
* Debug a **real trajectory you already logged**

Just tell me which one.


